In [6]:
import os
import json
import glob
import re
from tqdm import tqdm


In [3]:
def remove_space(text):
    # Remove multiple spaces
    text = ' '.join(text.split())
    # lines = text.split('\n')
    # for i,line in enumerate(lines):
        
    # lines[i] = lines[i].replace('  ', ' ')
    # Fix spacing around punctuation
    text = re.sub(r'\s+([.,!?])', r'\1', text)
    text = re.sub(r'([.,!?])\s+', r'\1 ', text)
    
    # Fix contractions
    text = re.sub(r'\s*\'\s*s\b', "'s", text)
    text = re.sub(r'\s*n\s*\'\s*t\b', "n't", text)
    text = re.sub(r'\s*\'\s*ve\b', "'ve", text)
    text = re.sub(r'\s*\'\s*re\b', "'re", text)
    text = re.sub(r'\s*\'\s*ll\b', "'ll", text)
    text = re.sub(r'\s*\'\s*d\b', "'d", text)
    text = re.sub(r'\s*\'\s*m\b', "'m", text)
    
    # Fix spaces around parentheses
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    
    # Remove spaces before and after text
    text = text.strip()
    text = text.replace('agent 0: ','')
    text = text.replace('agent 1: ','')
    return text


In [ ]:
# Get all json and jsonl files in data directory
json_files = glob.glob('../data/modified_data/ner/*.json*', recursive=True)

# Process each file
for file_path in json_files:
    print(f"Processing: {file_path}")
    
    try:
        # Read data based on file extension
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            if file_path.endswith('.jsonl'):
                # For jsonl files, read line by line and convert to json
                data = []
                for line in f:
                    try:
                        # Handle escaped characters by using raw string
                        line = line.replace('\\', '\\\\')
                        data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"Error processing line in {file_path}: {e}")
                        continue
            else:
                # For json files, load entire file
                content = f.read()
                content = content.replace('\\', '\\\\')
                data = json.loads(content)
        
        # ... rest of the processing code ...
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        continue    # Process all string values recursively
    def process_strings(obj):
        if isinstance(obj, str):
            return remove_space(obj)
        elif isinstance(obj, list):
            return [process_strings(item) for item in obj]
        elif isinstance(obj, dict):
            return {k: process_strings(v) for k, v in obj.items()}
        return obj
    
    processed_data = process_strings(data)
    # Save processed data back to file as json
    output_path = file_path.replace('.jsonl', '.json')
    with open(output_path, 'w') as f:
        json.dump(processed_data, f, indent=2)


In [ ]:
# Get all json and jsonl files in data directory
json_files = glob.glob('../data/modified_data/dialogue/*.json', recursive=True)

# Process each file
for file_path in json_files:
    print(f"Processing: {file_path}")
    
    # try:
    #     # Read data based on file extension
    #     with open(file_path, 'r') as f:
    #         if file_path.endswith('.json'):
    #             # For jsonl files, read line by line and convert to json
    #             data = []
    #             for line in f:
    #                 try:
    #                     # Handle escaped characters by using raw string
    #                     line = line.replace('\\', '\\\\')
    #                     data.append(json.loads(line))
    #                 except json.JSONDecodeError as e:
    #                     print(f"Error processing line in {file_path}: {e}")
    #                     continue
    #         else:
    #             # For json files, load entire file
    #             content = f.read()
    #             content = content.replace('\\', '\\\\')
    #             data = json.loads(content)
            
        # ... rest of the processing code ...
    # except Exception as e:
    #     print(f"Error processing file {file_path}: {e}")
    #     continue    # Process all string values recursively
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        continue    # Process all string values recursively

    def process_strings(obj):
        if isinstance(obj, str):
            return remove_space(obj)
        elif isinstance(obj, list):
            return [process_strings(item) for item in obj]
        elif isinstance(obj, dict):
            return {k: process_strings(v) for k, v in obj.items()}
        return obj
    
    processed_data = process_strings(data)
    
    # Save processed data back to file as json
    output_path = file_path.replace('.jsonl', '.json')
    with open(output_path, 'w') as f:
        json.dump(processed_data, f, indent=2)

In [ ]:
json_files = glob.glob('../data/modified_data/ner/*.json*', recursive=True)

# Process each file
for file_path in json_files:
    print(f"Processing: {file_path}")
    with open(file_path, 'r') as f:
        data = json.load(f)
    new_data = []
    flag = False
    for sample in data:
        if sample.get('Label') != None:
            flag = True
            label = sample['Label']
            new_label = []
            for pair in label:
                for key, value in pair.items():
                    item = {'text': key, 'value': value}
                    new_label.append(item)
            sample['label'] = new_label
            # print(sample['label'])
            del sample['Label']
            new_data.append(sample)
    if flag == True:
        print('finished')
        print(new_data[0])
        with open(file_path, 'w') as f:
            json.dump(new_data, f, indent=2)

In [ ]:
# import json
import glob
# import os
import random

# Load all JSON files from the specified directory
json_files = glob.glob('../data/modified_data/dialogue/*_100.json', recursive=True)

# Initialize a set to track unique modified_text entries

# Process each file
for file_path in json_files:
    print(file_path)
    if 'negation' not in file_path and 'grammatical_role' not in file_path:
        continue
    print(f"Loading: {file_path}")
    unique_modified_texts = set()
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    new_data = []  # Reset new_data for each file
    for sample in data:
        if isinstance(sample, list):
            modified_text = sample[1].get('modified_text')
            modified_label = sample[1].get('modified_label')
        else:
            modified_text = sample.get('modified_text')
            modified_label = sample.get('modified_label')
        
        if modified_label and modified_text and modified_text not in unique_modified_texts:
            unique_modified_texts.add(modified_text)
            new_data.append(sample)

    # Sample more data if we have less than 100 entries
    if len(new_data) < 100:
        # Load the original data again without the _100 suffix
        original_file_path = file_path.replace('_100', '')
        if 'capitalization' in original_file_path or 'punctuation' in original_file_path or 'typo' in original_file_path:
            continue
        with open(original_file_path, 'r') as f:
            original_data = json.load(f)
        
        # Filter out already included modified_texts
        if isinstance(sample, list):
            additional_samples = [s for s in original_data if s[1].get('modified_text') not in unique_modified_texts and s[1].get('modified_label') != 2]
        else:
            additional_samples = [s for s in original_data if s.get('modified_text') not in unique_modified_texts and s.get('modified_label') != 2]

        random.shuffle(additional_samples)  # Shuffle to get random samples

        # Add samples until we reach 100
        for sample in additional_samples:
            if len(new_data) >= 100:
                break
            new_data.append(sample)
            if isinstance(sample, list):
                unique_modified_texts.add(sample[1].get('modified_text'))
            else:
                unique_modified_texts.add(sample.get('modified_text'))

    # Save the deduplicated data back to a new JSON file for each original file
    with open(file_path, 'w') as f:
        json.dump(new_data, f, indent=2)

    print(f"Preserve: {len(new_data)}")


In [ ]:
# Load the active to passive data
active_to_passive_file_path = '../data/modified_data/coref/active_to_passive.json'
with open(active_to_passive_file_path, 'r') as f:
    active_to_passive_data = json.load(f)

# Replace modified_candidates with original_candidates
for sample in active_to_passive_data:
    sample['modified_candidates'] = sample['original_candidates']

# Save the modified data back to the same file or a new file
output_active_to_passive_path = '../data/modified_data/coref/active_to_passive.json'
with open(output_active_to_passive_path, 'w') as f:
    json.dump(active_to_passive_data, f, indent=2)

print(f"Modified active to passive data saved to: {output_active_to_passive_path}")


In [ ]:
# Load all JSON files in the specified directory
json_files = glob.glob('../data/modified_data/coref/*.json')

# Define the files to exclude
exclude_files = [
    'capitalization.json',
    'punctuation.json',
    'typo_bias.json',
    'grammatical_role.json',
    'negation.json'
]

# Process each JSON file
for file_path in json_files:
    if os.path.basename(file_path) in exclude_files:
        continue

    with open(file_path, 'r') as f:
        data = json.load(f)

    # Replace modified_pronoun with original_pronoun
    for sample in data:
        sample['modified_pronoun'] = sample['original_pronoun']

    # Save the modified data back to the same file
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=2)

    print(f"Modified pronouns in: {file_path}")


In [ ]:
import os
import json

# Initialize counters
negation_differences = 0
grammatical_role_differences = 0

# Load all JSON files in the specified directory
json_files = os.listdir('../data/modified_data/coref/')

# Process each JSON file
for file_name in json_files:
    if file_name in ['negation.json', 'grammatical_role.json']:
        with open(f'../data/modified_data/coref/{file_name}', 'r') as f:
            data = json.load(f)

        # Count differences in modified_label and original_label, ignoring label 2
        for sample in data:
            if sample['modified_label'] != sample['original_label'] and sample['modified_label'] != 2:
                if file_name == 'negation.json':
                    negation_differences += 1
                elif file_name == 'grammatical_role.json':
                    grammatical_role_differences += 1

print(f"Differences in negation: {negation_differences}")
print(f"Differences in grammatical_role: {grammatical_role_differences}")


In [ ]:
import pandas as pd

# Load the reference CSV file with indices
reference_df = pd.read_json('../preprocessing/train_dev_test_data/coref/test.json')

# Create a mapping of text to index for O(1) lookup
text_to_idx = {row['text']: idx for idx, row in reference_df.iterrows()}

# Load all JSON files in the specified directory
json_files = os.listdir('../data/modified_data/coref/')

# Process each JSON file
for file_name in json_files:
    file_path = f'../data/modified_data/coref/{file_name}'
    print(file_path)
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Add index to each sample based on exact text matching
    for sample in tqdm(data):
        if sample['original_text'] in text_to_idx:
            sample['index'] = int(text_to_idx[sample['original_text']])
        else:
            print(f"Warning: No exact match found for sample in {file_name}")
            
    # Save the modified data back to the same file
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=2)

    print(f"Added indices to: {file_path}")


In [ ]:
# Load the reference CSV file with indices for sentiment analysis
reference_df_sentiment = pd.read_csv('./results/sa/gpt4o-0shot-sst2.csv')

# Load all JSON files in the sentiment analysis directory
json_files_sentiment = os.listdir('../data/modified_data/sa/')

# Process each JSON file
for file_name in json_files_sentiment:
    file_path = f'../data/modified_data/sa/{file_name}'
    print(file_path)
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Add index to each sample based on fuzzy matching original_text
    for sample in tqdm(data):
        # Find best matching row in reference CSV using fuzzy matching
        best_match_ratio = 0
        best_match_idx = None
        
        for idx, ref_text in enumerate(reference_df_sentiment['text']):
            similarity = string_similarity(sample['original_text'], ref_text)
            if similarity > best_match_ratio:
                best_match_ratio = similarity
                best_match_idx = idx
        
        if best_match_ratio >= 0.9:
            sample['index'] = int(best_match_idx)
        else:
            print(f"Warning: No matching text found (threshold 0.9) for sample in {file_name}")
            # break
            
    # Save the modified data back to the same file
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=2)

    print(f"Added indices to: {file_path}")


In [ ]:
import pandas as pd

# Load the reference CSV file with indices
reference_df = pd.read_json('../preprocessing/train_dev_test_data/dialogue/test.json')

# Create a mapping of text to index for O(1) lookup
text_to_idx = {row['dialogue'][-1]: idx for idx, row in reference_df.iterrows()}

# Load only the specified JSON files
json_files = ['grammatical_role.json', 'negation.json', 
              'capitalization_100.json', 'grammatical_role_100.json', 'negation_100.json', 
              'punctuation_100.json', 'typo_bias_100.json']

# Process each JSON file
for file_name in json_files:
    file_path = f'../data/modified_data/dialogue/{file_name}'
    print(file_path)
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Add index to each sample based on exact text matching
    new_data = []
    for sample_tuple in tqdm(data):
        sample = sample_tuple  # Get the sample dict from tuple
        if sample['original_text'] in text_to_idx:
            idx = int(text_to_idx[sample['original_text']])
            new_data.append([idx, sample])  # Create new tuple with index first
        else:
            print(f"Warning: No exact match found for sample in {file_name}")
            
    # Save the modified data back to the same file
    with open(file_path, 'w') as f:
        json.dump(new_data, f, indent=2)

    print(f"Added indices to: {file_path}")


In [ ]:
# Load test.json and get sample with index 3301
with open('../preprocessing/train_dev_test_data/dialogue/test.json', 'r') as f:
    test_data = json.load(f)
    
sample_3301 = test_data[3301]
print(json.dumps(sample_3301, indent=2))


In [22]:
import difflib

In [ ]:
# Load the CSV file
csv_path = './results/dialogue/claude-3-5-sonnet-0shot-dialogue.csv'
df = pd.read_csv(csv_path)

# Load test.json for reference
with open('../preprocessing/train_dev_test_data/dialogue/test.json', 'r') as f:
    test_data = json.load(f)

# Create mapping of test dialogues to indices
test_dialogues = {sample['dialogue'][-1]: idx for idx, sample in enumerate(test_data)}

# Function to find best match using difflib
def find_best_match(text, candidates):
    best_ratio = 0
    best_match = None
    for candidate, idx in candidates.items():
        ratio = difflib.SequenceMatcher(None, text, candidate).ratio()
        if ratio > best_ratio:
            best_ratio = ratio
            best_match = (idx, ratio)
    return best_match

# Add index column based on matching
indices = []
for _, row in tqdm(df.iterrows()):
    dialog = row['dialog'].split('\n')[-1].replace('agent 0: ', '').replace('agent 1: ', '')  # Get last message
    match = find_best_match(dialog, test_dialogues)
    if match and match[1] > 0.9:  # Only keep matches with ratio > 0.9
        indices.append(match[0])
    else:
        print(f"No match found for {dialog}")
        indices.append(-1)  # Use -1 for no match found

# Add indices column to dataframe
df['index'] = indices

# Save updated CSV
df.to_csv(csv_path, index=False)

print(f"Added indices to {csv_path}")


In [ ]:
import os
import json
from tqdm import tqdm
import ast

# Get specific json files
json_dir = '../data/modified_data/ner'
target_files = ['coordinating_conjunction_100.json', 'grammatical_role_100.json', 'geographical_bias_100.json']

# Process each target file
for filename in target_files:
    filepath = os.path.join(json_dir, filename)
    
    # Load the JSON data
    with open(filepath, 'r') as f:
        data = json.load(f)
    
    # Process each item in the data
    for item in tqdm(data):
        # Convert label fields if they exist
        for field in ['label', 'modified_label', 'original_label']:
            if field in item and isinstance(item[field], list):
                # Handle original_label and modified_label format conversion
                if field in ['original_label', 'modified_label']:
                    # Convert list of list format to single list
                    if isinstance(item[field], list) and len(item[field]) > 0 and isinstance(item[field][0], list):
                        item[field] = item[field][0]
                        continue

                # Handle list of strings containing JSON
                for i, label in enumerate(item[field]):
                    if isinstance(label, str):
                        try:
                            # Handle both single dict strings and list of dict strings
                            if label.startswith('['):
                                # Parse list of dicts
                                # First try to parse as a Python literal
                                try:
                                    label_list = ast.literal_eval(label)
                                except:
                                    # If that fails, try cleaning and parsing each item individually
                                    label = label.strip('[]')
                                    label_list = [l.strip() for l in label.split(',')]
                                
                                # Handle each dict in the list separately
                                parsed_labels = []
                                for l in label_list:
                                    try:
                                        # Try parsing as Python literal first
                                        parsed = ast.literal_eval(str(l))
                                        parsed_labels.append(parsed)
                                    except:
                                        try:
                                            # Try JSON parsing with quote replacement
                                            cleaned = str(l).replace("'", '"').replace('\\"', '"')
                                            parsed = json.loads(cleaned)
                                            parsed_labels.append(parsed)
                                        except:
                                            # Last resort - manual string cleaning
                                            cleaned = str(l).replace("\\", "").replace("'", '"')
                                            cleaned = cleaned.replace('"{', '{').replace('}"', '}')
                                            parsed = json.loads(cleaned)
                                            parsed_labels.append(parsed)
                                            
                                item[field][i] = parsed_labels
                            else:
                                # Parse single dict
                                try:
                                    item[field][i] = ast.literal_eval(label)
                                except:
                                    item[field][i] = json.loads(label.replace("'", '"'))
                        except Exception as e:
                            print(f"Failed to convert {field}[{i}] in file {filename}")
                            print(f"Value: {label}")
                            print(f"Error: {str(e)}")
                            continue
                
    # Save the updated data back to file
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=2)
        
print("Completed processing target JSON files")


In [ ]:
import pandas as pd

# Load the reference CSV file with indices
reference_df = pd.read_json('../preprocessing/train_dev_test_data/ner/fewnerd_sample_test.json')

# Create a mapping of text to index for O(1) lookup
text_to_idx = {row['text']: idx for idx, row in reference_df.iterrows()}

# Load all JSON files in the specified directory
json_files = [f for f in os.listdir('../data/modified_data/ner/') if f.endswith('.json')]
# Filter to only process specific files
# target_files = ['grammatical_role_100.json', 'coordinating_conjunction_100.json', 'geographical_bias_100.json']
# target_files = ['derivation.json']

# json_files = [f for f in json_files if f in target_files]

# Process each JSON file
for file_name in json_files:
    file_path = f'../data/modified_data/ner/{file_name}'
    print(file_path)
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Add index to each sample based on exact text matching
    for sample in tqdm(data):
        # Find best match using difflib ratio with threshold 0.9
        if 'index' in sample and 'original_text' in sample and 'original_label' in sample:
            continue
        if 'original_text' not in sample:
            sample['original_text'] = sample['text']
        if 'original_label' not in sample:
            sample['original_label'] = sample['label']
        best_match = None
        best_ratio = 0
        for ref_text in text_to_idx:
            # original_text = sample['original_text'] if 'original_text' in sample else sample['text']
            
            ratio = difflib.SequenceMatcher(None, sample['original_text'], ref_text).ratio()
            if ratio > 0.9 and ratio > best_ratio:
                best_ratio = ratio
                best_match = ref_text
                
        if best_match:
            sample['index'] = int(text_to_idx[best_match])
        else:
            print(sample['original_text'])
            print(f"Warning: No exact match found for sample in {file_name}")
            
    # Save the modified data back to the same file
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=2)

    print(f"Added indices to: {file_path}")


In [ ]:
import pandas as pd
import json
import os
from tqdm import tqdm
import difflib

# Load the reference data with original labels
reference_df = pd.read_json('../preprocessing/train_dev_test_data/ner/fewnerd_sample_test.json')

# Create mapping of text to original labels
text_to_labels = {row['text']: row['label'] for _, row in reference_df.iterrows()}

# Process each JSON file
json_files = [f for f in os.listdir('../preprocessing/data_for_phase2/ner/') if f.endswith('.json')]
# Filter to only include derivation.json
json_files = ['derivation.json']

for file_name in json_files:
    file_path = f'../preprocessing/data_for_phase2/ner/{file_name}'
    
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Add original labels to each sample based on text matching
    for sample in tqdm(data):
        # Find best match using difflib ratio with threshold 0.9
        best_match = None
        best_ratio = 0
        for ref_text in text_to_labels:
            ratio = difflib.SequenceMatcher(None, sample['original_text'], ref_text).ratio()
            if ratio > 0.9 and ratio > best_ratio:
                best_ratio = ratio
                best_match = ref_text
                
        if best_match:
            sample['original_label'] = text_to_labels[best_match]
        else:
            print(sample['original_text'])
            print(f"Warning: No match found for sample in {file_name}")
            
    # Save the modified data back to the same file
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=2)

    print(f"Added original labels to: {file_path}")


In [63]:
import numpy as np

In [ ]:
# Load the comparison CSV
comparison_df = pd.read_csv('/Users/hungcoreft/workspace/ood-modification/preprocessing/scripts/grammatical_role_coref_comparison.csv')

# Drop rows with Keep == 0 and drop Changes column
filtered_df = comparison_df[comparison_df['Keep'].isin([1, np.nan])].drop('Changes', axis=1)
print(len(filtered_df))
# Load the original JSON file
json_path = '../data/modified_data/coref/grammatical_role_100.json'
with open(json_path, 'r') as f:
    data = json.load(f)

# Create mapping of indices to data entries
data_dict = {item['index']: item for item in data}

# Update data based on filtered rows
for _, row in filtered_df.iterrows():
    # print(row)
    idx = row['index']
    if idx in data_dict:
        # data_dict[idx]['original_text'] = row['original_text']
        # data_dict[idx]['modified_text'] = row['modified_text']
        # data_dict[idx]['original_label'] = row['original_label']
        data_dict[idx]['modified_label'] = row['modified_label']
        data_dict[idx]['modified_pronoun'] = row['modified_pronoun']
        data_dict[idx]['modified_candidates'] = ast.literal_eval(row['modified_candidates'])

# Convert back to list and maintain only filtered indices
filtered_data = [data_dict[idx] for idx in filtered_df['index']]

# Save filtered data back to JSON
with open(json_path, 'w') as f:
    json.dump(filtered_data, f, indent=2)

print(f"Filtered and saved {len(filtered_data)} entries to {json_path}")


In [ ]:
# Load the comparison CSV
comparison_df = pd.read_csv('/Users/hungcoreft/workspace/ood-modification/preprocessing/scripts/grammatical_role_ner_comparison.csv')
filtered_df = comparison_df
# Drop rows with Keep == 0 and drop Changes column
# filtered_df = comparison_df[comparison_df['Keep'].isin([1, np.nan])].drop('Changes', axis=1)
print(len(filtered_df))
# Load the original JSON file
json_path = '../data/modified_data/ner/grammatical_role_100.json'
with open(json_path, 'r') as f:
    data = json.load(f)

# Create mapping of indices to data entries
data_dict = {item['index']: item for item in data}

# Update data based on filtered rows
for _, row in filtered_df.iterrows():
    # print(row)
    idx = row['index']
    if idx in data_dict:
        # data_dict[idx]['original_text'] = row['original_text']
        # data_dict[idx]['modified_text'] = row['modified_text']
        # data_dict[idx]['original_label'] = row['original_label']
        modified_label = ast.literal_eval(row['modified_label'])
        data_dict[idx]['modified_label'] = modified_label
        # data_dict[idx]['modified_pronoun'] = row['modified_pronoun']
        # data_dict[idx]['modified_candidates'] = row['modified_candidates']

# Convert back to list and maintain only filtered indices
filtered_data = [data_dict[idx] for idx in filtered_df['index']]

# Save filtered data back to JSON
with open(json_path, 'w') as f:
    json.dump(filtered_data, f, indent=2)

print(f"Filtered and saved {len(filtered_data)} entries to {json_path}")


In [ ]:
# Load the comparison CSV
comparison_df = pd.read_csv('./derivation_coref.csv')

# Drop rows with Keep == 0 and drop Changes column
filtered_df = comparison_df[comparison_df['Keep'].isin(['Yes', np.nan])].drop('Changes', axis=1)
print(len(filtered_df))
# Load the original JSON file
json_path = '../data/modified_data/coref/derivation_100.json'
with open(json_path, 'r') as f:
    data = json.load(f)

# Create mapping of indices to data entries
data_dict = {item['index']: item for item in data}

# Update data based on filtered rows
for _, row in filtered_df.iterrows():
    # print(row)
    idx = row['index']
    if idx in data_dict:
        # data_dict[idx]['original_text'] = row['original_text']
        data_dict[idx]['modified_text'] = row['modified_text']
        # data_dict[idx]['original_label'] = row['original_label']
        data_dict[idx]['modified_label'] = row['modified_label']
        data_dict[idx]['modified_pronoun'] = row['modified_pronoun']
        data_dict[idx]['modified_candidates'] = ast.literal_eval(row['modified_candidates'])

# Convert back to list and maintain only filtered indices
filtered_data = [data_dict[idx] for idx in filtered_df['index']]

# Save filtered data back to JSON
with open(json_path, 'w') as f:
    json.dump(filtered_data, f, indent=2)

print(f"Filtered and saved {len(filtered_data)} entries to {json_path}")


In [ ]:
# Load the comparison CSV
comparison_df = pd.read_csv('./derivation_sentiment.csv')

# Drop rows with Keep == 0 and drop Changes column
filtered_df = comparison_df[comparison_df['Keep'].isin(['Yes', np.nan])].drop('Changes', axis=1)
print(len(filtered_df))
# Load the original JSON file
json_path = '../data/modified_data/sa/derivation_100.json'
with open(json_path, 'r') as f:
    data = json.load(f)

# Create mapping of indices to data entries
data_dict = {item['index']: item for item in data}

# Update data based on filtered rows
for _, row in filtered_df.iterrows():
    # print(row)
    idx = row['index']
    if idx in data_dict:
        # data_dict[idx]['original_text'] = row['original_text']
        data_dict[idx]['modified_text'] = row['modified_text']
        # data_dict[idx]['original_label'] = row['original_label']
        # data_dict[idx]['modified_label'] = row['modified_label']
        # data_dict[idx]['modified_pronoun'] = row['modified_pronoun']
        # data_dict[idx]['modified_candidates'] = row['modified_candidates']

# Convert back to list and maintain only filtered indices
filtered_data = [data_dict[idx] for idx in filtered_df['index']]
print(filtered_data[0])
# Save filtered data back to JSON
with open(json_path, 'w') as f:
    json.dump(filtered_data, f, indent=2)

print(f"Filtered and saved {len(filtered_data)} entries to {json_path}")


In [ ]:
# Load the comparison CSV
comparison_df = pd.read_csv('./derivation_ner.csv')

# Drop rows with Keep == 0 and drop Changes column
filtered_df = comparison_df[comparison_df['Keep'].isin(['Yes', np.nan])].drop('Changes', axis=1)
print(len(filtered_df))
# Load the original JSON file

json_path = '../preprocessing/train_dev_test_data/ner/fewnerd_sample_test.json'
with open(json_path, 'r') as f:
    data = json.load(f)

# Create mapping of indices to data entries
data_dict = {i: item for i, item in enumerate(data)}


# Update data based on filtered rows
for _, row in filtered_df.iterrows():
    # print(row)
    idx = row['index']
    if idx in data_dict:
        # data_dict[idx]['original_text'] = row['original_text']
        data_dict[idx]['modified_text'] = row['modified_text']
        # data_dict[idx]['original_label'] = row['original_label']
        modified_label = ast.literal_eval(row['modified_label'])
        data_dict[idx]['modified_label'] = modified_label
        # data_dict[idx]['modified_pronoun'] = row['modified_pronoun']
        # data_dict[idx]['modified_candidates'] = row['modified_candidates']

# Convert back to list and maintain only filtered indices
filtered_data = [data_dict[idx] for idx in filtered_df['index']]
print(filtered_data[1])
# Save filtered data back to JSON
json_path = '../data/modified_data/ner/derivation_100.json'
with open(json_path, 'w') as f:
    json.dump(filtered_data, f, indent=2)

print(f"Filtered and saved {len(filtered_data)} entries to {json_path}")


In [ ]:
# Load the comparison CSV
comparison_df = pd.read_csv('/Users/hungcoreft/workspace/ood-modification/preprocessing/scripts/grammatical_role_dialogue_comparison.csv')

# Drop rows with Keep == 0 and drop Changes column
print(len(comparison_df))

filtered_df = comparison_df[comparison_df['Keep'].isin([1, np.nan])].drop('Changes', axis=1)
print(len(filtered_df))
# Load the original JSON file
json_path = '../data/modified_data/dialogue/grammatical_role.json'
with open(json_path, 'r') as f:
    data = json.load(f)

# Create mapping of indices to data entries
data_dict = {index: item for index, item in data}

# Update data based on filtered rows
for _, row in filtered_df.iterrows():
    # print(row)
    idx = row['index']
    if idx in data_dict:
        # data_dict[idx]['original_text'] = row['original_text']
        # data_dict[idx]['modified_text'] = row['modified_text']
        # data_dict[idx]['original_label'] = row['original_label']
        data_dict[idx]['modified_label'] = row['modified_label']
        # data_dict[idx]['modified_pronoun'] = row['modified_pronoun']
        # data_dict[idx]['modified_candidates'] = row['modified_candidates']

# Convert back to list and maintain only filtered indices
filtered_data = [[idx,data_dict[idx]] for idx in filtered_df['index']]
# print(filtered_data[0])
# Save filtered data back to JSON
with open(json_path, 'w') as f:
    json.dump(filtered_data, f, indent=2)

print(f"Filtered and saved {len(filtered_data)} entries to {json_path}")


In [ ]:
# Load the comparison CSV
comparison_df = pd.read_csv('./derivation_dialog.csv')

# Drop rows with Keep == 0 and drop Changes column
print(len(comparison_df))

filtered_df = comparison_df[comparison_df['Keep'].isin(["Yes", np.nan])].drop('Changes', axis=1)
print(len(filtered_df))
# Load the original JSON file

# Load test data to get dialogue indices
test_path = '../preprocessing/train_dev_test_data/dialogue/test.json'
with open(test_path, 'r') as f:
    test_data = json.load(f)

# Create mapping of last turn text to index
label_mapping = {
    'is_contradiction': 1,
    'no_contradiction': 0
}
samples = []
for _, row in filtered_df.iterrows():
    # print(row)
    idx = row['index']
    sample = test_data[idx]
    last_turn = sample['turns'][-1]['text']
    sample['modified_text'] = row['modified_text']
    # samples.append(sample)
    sample['dialog_context'] = [turn['text'] for turn in sample['turns'][:-1]]
    sample['original_text'] = sample['turns'][-1]['text']
    sample['label'] = label_mapping[sample['label']]
    item = [idx, sample]
    samples.append(item)
# Convert back to list and maintain only filtered indices
print(len(samples))
print(samples[1])
# print(filtered_data[0])
# Save filtered data back to JSON
json_path = '../data/modified_data/dialogue/derivation_100.json'
with open(json_path, 'w') as f:
    json.dump(samples, f, indent=2)

print(f"Filtered and saved {len(samples)} entries to {json_path}")


In [ ]:
# Load both JSON files
file1_path = '../data/modified_data/ner/grammatical_role_100.json'
file2_path = '../data/modified_data/ner/grammatical_role_100_compare.json'

with open(file1_path, 'r') as f1, open(file2_path, 'r') as f2:
    data1 = json.load(f1)
    data2 = json.load(f2)

# Create sets of modified texts from both files
texts1 = {item['modified_text'] for item in data1}
texts2 = {item['modified_text'] for item in data2}

# Print total counts
print(f"Total texts in file 1: {len(texts1)}")
print(f"Total texts in file 2: {len(texts2)}")
print(f"Texts in both files: {len(texts1.intersection(texts2))}")

# Find texts present in only one file
only_in_file1 = texts1 - texts2
only_in_file2 = texts2 - texts1

# Print texts present only in file 1
print(f"\nTexts ONLY in file 1 ({len(only_in_file1)}):")
for text in only_in_file1:
    print(f"\nText: {text}")

# Print texts present only in file 2  
print(f"\nTexts ONLY in file 2 ({len(only_in_file2)}):")
for text in only_in_file2:
    print(f"\nText: {text}")

# Print some examples of texts in both files
common_texts = texts1.intersection(texts2)
print(f"\nExample texts present in BOTH files (showing first 3):")
for text in list(common_texts)[:3]:
    print(f"\nText: {text}")


In [93]:
import csv
with open('/Users/hungcoreft/workspace/ood-modification/preprocessing/scripts/coordinating_conjunction_ner_comparison_detailed.csv', 'r') as f:
    csv_reader = csv.DictReader(f)
    comparison_data = [row for row in csv_reader]
for row in comparison_data:
    row['index'] = row['index']
    row['original_text'] = row['original_text']
    row['modified_text'] = row['modified_text']
    row['modified_label'] = ast.literal_eval(row['modified_label'])
    row['original_label'] = ast.literal_eval(row['original_label'])
json.dump(comparison_data, open('coordinating_conjunction_ner_compare.json', 'w'), indent=2)



In [ ]:
file1_path = '../data/modified_data/ner/coordinating_conjunction_100.json'
file2_path = '../data/modified_data/ner/coordinating_conjunction_100 copy.json'

with open(file1_path, 'r') as f1, open(file2_path, 'r') as f2:
    data1 = json.load(f1)
    data2 = json.load(f2)

# Create sets of modified texts from both files
texts1 = {item['modified_text'] for item in data1}
texts2 = {item['modified_text'] for item in data2}

# Print total counts
print(f"Total texts in file 1: {len(texts1)}")
print(f"Total texts in file 2: {len(texts2)}")
print(f"Texts in both files: {len(texts1.intersection(texts2))}")

# Find texts present in only one file
only_in_file1 = texts1 - texts2
only_in_file2 = texts2 - texts1

# Print texts present only in file 1
print(f"\nTexts ONLY in file 1 ({len(only_in_file1)}):")
for text in only_in_file1:
    print(f"\nText: {text}")

# Print texts present only in file 2  
print(f"\nTexts ONLY in file 2 ({len(only_in_file2)}):")
for text in only_in_file2:
    print(f"\nText: {text}")

# Print some examples of texts in both files
common_texts = texts1.intersection(texts2)
print(f"\nExample texts present in BOTH files (showing first 3):")
for text in list(common_texts)[:3]:
    print(f"\nText: {text}")


In [ ]:
# Read both CSV files
old_file = pd.read_csv('/Users/hungcoreft/workspace/ood-modification/preprocessing/scripts/derivation_old_ner_comparison_detailed.csv')
new_file = pd.read_csv('/Users/hungcoreft/workspace/ood-modification/preprocessing/scripts/derivation_ner_comparison_detailed.csv')

print(len(old_file))
print(len(new_file))
# Create sets of modified texts from both files
old_texts = set(old_file['modified_text'])
new_texts = set(new_file['modified_text'])

print(len(old_texts))
print(len(new_texts))
# Find texts present only in old file
only_in_old = old_texts - new_texts

# Get rows that are only in old file
samples_df = old_file[old_file['modified_text'].isin(only_in_old)]

# Save to CSV
samples_df.to_csv('derivation_ner_comparison_samples.csv', index=False)


In [ ]:
import json
import random

# Read the existing data from data_after_phase2
with open('../data/modified_data/dialogue/derivation.json', 'r') as f:
    existing_data = json.load(f)

existing_data = [sample[1] for sample in existing_data]
# Create set of existing modified texts
existing_texts = set(item['modified_text'] for item in existing_data)

# Read the filtered data
with open('../preprocessing/filtered_data/dialogue/derivation.json', 'r') as f:
    filtered_data = json.load(f)

print(filtered_data[0])
# Get items from filtered data that aren't in existing data
new_items = [item for item in filtered_data if item['modified_text'] not in existing_texts]

# Sample 50 items randomly
# Sample 50 items randomly
sampled_items = random.sample(new_items, min(100, len(new_items)))

# Load test data to get indices
with open('../preprocessing/train_dev_test_data/dialogue/test.json', 'r') as f:
    test_data = json.load(f)

# Create mapping of test indices
from difflib import SequenceMatcher

# Create mapping of test indices using fuzzy matching
test_indices = {item['dialogue'][-1]: i for i, item in enumerate(test_data)}

# Map sampled items to test indices using fuzzy matching
matched_items = []
for item in sampled_items:
    for test_text, idx in test_indices.items():
        if SequenceMatcher(None, item['modified_text'], test_text).ratio() > 0.9:
            item['index'] = idx
            matched_items.append(item)
            break

sampled_items = matched_items[:50]
# Save sampled items to a new JSON file
# Find differences between original and modified text
# Convert sampled items to DataFrame
df = pd.DataFrame(sampled_items)

# Add changes column by comparing original and modified text
changes = []
for _, row in df.iterrows():
    original = row['original_text'].split()
    modified = row['modified_text'].split()
    
    item_changes = []
    for o, m in zip(original, modified):
        if o != m:
            item_changes.append(f"{o} -> {m}")
    
    changes.append(', '.join(item_changes))

df['Changes'] = changes

# Keep only desired columns
df = df[['index', 'original_text', 'modified_text', 'Changes']]

# Save to CSV
df.to_csv('derivation_samples_dialogue.csv', index=False)
print(f"Sampled {len(sampled_items)} new items")


In [189]:
from scipy import stats

In [191]:
def highlight_drops_and_significance(row):
    colors = [''] * len(row)
    if row['original_res'] > row['modified_res']:
        colors = ['background-color: red'] * len(row)
        # If p-value < 0.05, add bold text
        if 'p_value' in row and row['p_value'] is not None and row['p_value'] < 0.05:
            colors = ['background-color: red; font-weight: bold'] * len(row)
    return colors



In [194]:
# Load the detailed results for negation for each model
sonnet_detailed = pd.read_csv('results/dialogue/claude-3-5-sonnet-0shot-negation_100.csv')
gpt4_detailed = pd.read_csv('results/dialogue/gpt4o-0shot-negation_100.csv')
llama_detailed = pd.read_csv('results/dialogue/llama-0shot-negation_100.csv')

# Calculate accuracy for original and modified examples by type for each model
type_results = []
for type_name in sonnet_detailed['type'].unique():
    # Get data for each model for this type
    sonnet_data = sonnet_detailed[sonnet_detailed['type'] == type_name]
    gpt4_data = gpt4_detailed[gpt4_detailed['type'] == type_name]
    llama_data = llama_detailed[llama_detailed['type'] == type_name]
    
    # Calculate accuracies for each model
    models_original_acc = []
    models_modified_acc = []
    
    for data in [sonnet_data, gpt4_data, llama_data]:
        original_acc = (data['original_pred'] == data['original_label']).sum() / len(data)
        modified_acc = (data['modified_pred'] == data['modified_label']).sum() / len(data)
        models_original_acc.append(original_acc)
        models_modified_acc.append(modified_acc)
    
    # Average across models
    avg_original_acc = sum(models_original_acc) / len(models_original_acc)
    avg_modified_acc = sum(models_modified_acc) / len(models_modified_acc)
    
    # Calculate differences
    difference = -(avg_original_acc - avg_modified_acc)
    pct_difference = -round((avg_original_acc - avg_modified_acc) / avg_original_acc * 100, 2)

    # Perform t-test on combined results
    all_original_correct = []
    all_modified_correct = []
    for data in [sonnet_data, gpt4_data, llama_data]:
        all_original_correct.extend((data['original_pred'] == data['original_label']).astype(int))
        all_modified_correct.extend((data['modified_pred'] == data['modified_label']).astype(int))
    
    _, p_value = stats.ttest_rel(all_original_correct, all_modified_correct)
    
    type_results.append({
        'type': type_name,
        'num_samples': len(sonnet_data),  # Total samples across all models
        'original_res': round(avg_original_acc * 100, 2),
        'modified_res': round(avg_modified_acc * 100, 2),
        'difference': round(difference * 100, 2),
        'pct_difference': round(pct_difference, 2),
        'p_value': p_value
    })

# Create and display type-based results dataframe
type_results_df = pd.DataFrame(type_results)

# Apply the same styling as before
styled_type_results = type_results_df.round(2).style.apply(highlight_drops_and_significance, axis=1)
styled_type_results
type_results_df.to_csv('dialog_negation_type_results.csv', index=False)



In [197]:
# Load the detailed results for sentiment analysis for each model
sonnet_detailed = pd.read_csv('results/sa/claude-3-5-sonnet-0shot-negation_100.csv')
gpt4_detailed = pd.read_csv('results/sa/gpt4o-0shot-negation_100.csv')
llama_detailed = pd.read_csv('results/sa/llama-0shot-negation_100.csv')

# Calculate accuracy for original and modified examples by type for each model
type_results = []
for type_name in sonnet_detailed['type'].unique():
    # Get data for each model for this type
    sonnet_data = sonnet_detailed[sonnet_detailed['type'] == type_name]
    gpt4_data = gpt4_detailed[gpt4_detailed['type'] == type_name]
    llama_data = llama_detailed[llama_detailed['type'] == type_name]
    
    # Calculate accuracies for each model
    models_original_acc = []
    models_modified_acc = []
    
    for data in [sonnet_data, gpt4_data, llama_data]:
        original_acc = (data['original_pred'] == data['original_label']).sum() / len(data)
        modified_acc = (data['modified_pred'] == data['modified_label']).sum() / len(data)
        models_original_acc.append(original_acc)
        models_modified_acc.append(modified_acc)
    
    # Average across models
    avg_original_acc = sum(models_original_acc) / len(models_original_acc)
    avg_modified_acc = sum(models_modified_acc) / len(models_modified_acc)
    
    # Calculate differences
    difference = -(avg_original_acc - avg_modified_acc)
    pct_difference = -round((avg_original_acc - avg_modified_acc) / avg_original_acc * 100, 2)

    # Perform t-test on combined results
    all_original_correct = []
    all_modified_correct = []
    for data in [sonnet_data, gpt4_data, llama_data]:
        all_original_correct.extend((data['original_pred'] == data['original_label']).astype(int))
        all_modified_correct.extend((data['modified_pred'] == data['modified_label']).astype(int))
    
    _, p_value = stats.ttest_rel(all_original_correct, all_modified_correct)
    
    type_results.append({
        'type': type_name,
        'num_samples': len(sonnet_data),  # Total samples across all models
        'original_res': round(avg_original_acc * 100, 2),
        'modified_res': round(avg_modified_acc * 100, 2),
        'difference': round(difference * 100, 2),
        'pct_difference': round(pct_difference, 2),
        'p_value': p_value
    })

# Create and display type-based results dataframe
type_results_df = pd.DataFrame(type_results)

# Apply the same styling as before
styled_type_results = type_results_df.round(2).style.apply(highlight_drops_and_significance, axis=1)
styled_type_results
type_results_df.to_csv('sentiment_negation_type_results.csv', index=False)


In [198]:
# Load the detailed results for sentiment analysis for each model
sonnet_detailed = pd.read_csv('results/coref/claude-3-5-sonnet-0shot-negation_100.csv')
gpt4_detailed = pd.read_csv('results/coref/gpt4o-0shot-negation_100.csv')
llama_detailed = pd.read_csv('results/coref/llama-0shot-negation_100.csv')

# Calculate accuracy for original and modified examples by type for each model
type_results = []
for type_name in sonnet_detailed['type'].unique():
    # Get data for each model for this type
    sonnet_data = sonnet_detailed[sonnet_detailed['type'] == type_name]
    gpt4_data = gpt4_detailed[gpt4_detailed['type'] == type_name]
    llama_data = llama_detailed[llama_detailed['type'] == type_name]
    
    # Calculate accuracies for each model
    models_original_acc = []
    models_modified_acc = []
    
    for data in [sonnet_data, gpt4_data, llama_data]:
        original_acc = (data['original_pred'] == data['original_label']).sum() / len(data)
        modified_acc = (data['modified_pred'] == data['modified_label']).sum() / len(data)
        models_original_acc.append(original_acc)
        models_modified_acc.append(modified_acc)
    
    # Average across models
    avg_original_acc = sum(models_original_acc) / len(models_original_acc)
    avg_modified_acc = sum(models_modified_acc) / len(models_modified_acc)
    
    # Calculate differences
    difference = -(avg_original_acc - avg_modified_acc)
    pct_difference = -round((avg_original_acc - avg_modified_acc) / avg_original_acc * 100, 2)

    # Perform t-test on combined results
    all_original_correct = []
    all_modified_correct = []
    for data in [sonnet_data, gpt4_data, llama_data]:
        all_original_correct.extend((data['original_pred'] == data['original_label']).astype(int))
        all_modified_correct.extend((data['modified_pred'] == data['modified_label']).astype(int))
    
    _, p_value = stats.ttest_rel(all_original_correct, all_modified_correct)
    
    type_results.append({
        'type': type_name,
        'num_samples': len(sonnet_data),  # Total samples across all models
        'original_res': round(avg_original_acc * 100, 2),
        'modified_res': round(avg_modified_acc * 100, 2),
        'difference': round(difference * 100, 2),
        'pct_difference': round(pct_difference, 2),
        'p_value': p_value
    })

# Create and display type-based results dataframe
type_results_df = pd.DataFrame(type_results)

# Apply the same styling as before
styled_type_results = type_results_df.round(2).style.apply(highlight_drops_and_significance, axis=1)
styled_type_results
type_results_df.to_csv('coreference_negation_type_results.csv', index=False)


In [203]:
def calculate_f1_ent(gold_entities, predicted_entities):
    """
    Calculates the F1 score given the true labels and predicted labels.
    """
    # print("Input types:")
    # print(f"gold_entities type: {type(gold_entities)}")
    # print(f"predicted_entities type: {type(predicted_entities)}")
    
    if predicted_entities is None:
        return 0.0, 0.0, 0.0

    true_entities = {}
    pred_entities = {}
    
    # Convert to empty list if NaN
    def handle_nan(entities):
        # print(f"Handling NaN for: {type(entities)}")
        # If it's already a list, return it as is
        if isinstance(entities, list):
            return entities
        # Handle pandas/numpy types
        if isinstance(entities, (pd.Series, np.ndarray)):
            nan_check = pd.isna(entities)
            if isinstance(nan_check, (pd.Series, np.ndarray)):
                if nan_check.any():
                    return "[]"
            elif nan_check:
                return "[]"
        # Handle single values
        elif pd.isna(entities):
            return "[]"
        return entities

    # Handle NaN cases
    gold_entities = handle_nan(gold_entities)
    predicted_entities = handle_nan(predicted_entities)
            
    # Parse strings if needed
    if isinstance(gold_entities, str):
        gold_entities = ast.literal_eval(gold_entities)
    if isinstance(predicted_entities, str):
        predicted_entities = ast.literal_eval(predicted_entities)

    # Process gold entities
    # print(gold_entities)
    for entity in gold_entities:
        # print(entity)
        if isinstance(entity, str):
            entity = ast.literal_eval(entity)
        if entity.get('text') is not None:
            true_entities[entity['text']] = entity['value']
        else:
            for key, value in entity.items():
                true_entities[key] = value
    
    # Process predicted entities
    for entity in predicted_entities:
        if isinstance(entity, str):
            entity = ast.literal_eval(entity)
        if entity.get('text') is not None:  
            pred_entities[entity['text']] = entity['value']
        else:
            for key, value in entity.items():
                pred_entities[key] = value

    # Calculate metrics
    true_positives = sum(1 for text in true_entities if text in pred_entities and true_entities[text] == pred_entities[text])
    false_positives = sum(1 for text in pred_entities if text not in true_entities)
    false_negatives = sum(1 for text in true_entities if text not in pred_entities)

    if true_positives == 0:
        return 0.0, 0.0, 0.0

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score

In [206]:
# Load the detailed results for sentiment analysis for each model
sonnet_detailed = pd.read_csv('results/ner/claude-0shot-negation_100.csv')
gpt4_detailed = pd.read_csv('results/ner/gpt4o-0shot-negation_100.csv')
llama_detailed = pd.read_csv('results/ner/llama-0shot-negation_100.csv')
def convert_string_to_entities(entity_str):
    """Convert string representation of entities to proper format"""
    if isinstance(entity_str, str):
        try:
            # Convert string to list of dicts
            entities = ast.literal_eval(entity_str)
            # Handle nested lists by flattening
            if isinstance(entities, list):
                # Handle double nested lists
                if len(entities) > 0 and isinstance(entities[0], list):
                    entities = entities[0]
                # Handle list of dicts with text/value format
                if len(entities) > 0 and isinstance(entities[0], dict):
                    # Handle format with text/value keys
                    if 'text' in entities[0]:
                        return entities
                    # Handle format with single key-value pair
                    if len(entities[0]) == 1:
                        converted = []
                        for e in entities:
                            for text, value in e.items():
                                converted.append({'text': text, 'value': value})
                        return converted
                    # Handle format with multiple key-value pairs
                    converted = []
                    for e in entities:
                        for text, value in e.items():
                            if isinstance(value, str):
                                converted.append({'text': text, 'value': value})
                    return converted
            return entities
        except:
            return []
    return entity_str

# Calculate accuracy for original and modified examples by type for each model
type_results = []
for type_name in sonnet_detailed['type'].unique():
    # Get data for each model for this type
    sonnet_data = sonnet_detailed[sonnet_detailed['type'] == type_name]
    gpt4_data = gpt4_detailed[gpt4_detailed['type'] == type_name]
    llama_data = llama_detailed[llama_detailed['type'] == type_name]
    

    # Calculate accuracies for each model
    models_original_f1 = []
    models_modified_f1 = []
    
    for data in [sonnet_data, gpt4_data, llama_data]:
        # Calculate F1 for each row then average
        original_f1s = []
        modified_f1s = []
        
        for _, row in data.iterrows():
            original_pred_ents = convert_string_to_entities(row['original_pred'])
            original_label_ents = convert_string_to_entities(row['original_label'])
            original_f1 = calculate_f1_ent(original_pred_ents, original_label_ents)
            original_f1s.append(original_f1)
            
            modified_pred_ents = convert_string_to_entities(row['modified_pred'])
            modified_label_ents = convert_string_to_entities(row['modified_label'])
            modified_f1 = calculate_f1_ent(modified_pred_ents, modified_label_ents)
            modified_f1s.append(modified_f1)
        
        models_original_f1.append(np.mean(original_f1s))
        models_modified_f1.append(np.mean(modified_f1s))
    # Average across models
    avg_original_f1 = sum(models_original_f1) / len(models_original_f1)
    avg_modified_f1 = sum(models_modified_f1) / len(models_modified_f1)
    
    # Calculate differences
    difference = -(avg_original_f1 - avg_modified_f1)
    pct_difference = -round((avg_original_f1 - avg_modified_f1) / avg_original_f1 * 100, 2)

    # Perform t-test on combined results
    all_original_f1s = []
    all_modified_f1s = []
    for data in [sonnet_data, gpt4_data, llama_data]:
        for _, row in data.iterrows():
            original_pred_ents = convert_string_to_entities(row['original_pred'])
            original_label_ents = convert_string_to_entities(row['original_label'])
            original_f1 = calculate_f1_ent(original_pred_ents, original_label_ents)
            all_original_f1s.append(original_f1)
            
            modified_pred_ents = convert_string_to_entities(row['modified_pred'])
            modified_label_ents = convert_string_to_entities(row['modified_label'])
            modified_f1 = calculate_f1_ent(modified_pred_ents, modified_label_ents)
            all_modified_f1s.append(modified_f1)
    
    _, p_value = stats.ttest_rel(all_original_f1s, all_modified_f1s)
    
    type_results.append({
        'type': type_name,
        'num_samples': len(sonnet_data),  # Total samples across all models
        'original_res': round(avg_original_f1 * 100, 2),
        'modified_res': round(avg_modified_f1 * 100, 2),
        'difference': round(difference * 100, 2),
        'pct_difference': round(pct_difference, 2),
        'p_value': p_value
    })

# Create and display type-based results dataframe
type_results_df = pd.DataFrame(type_results)

# Apply the same styling as before
styled_type_results = type_results_df.round(2).style.apply(highlight_drops_and_significance, axis=1)
styled_type_results
type_results_df.to_csv('ner_negation_type_results.csv', index=False)


In [241]:
# Load OOD results for coreference resolution
ood_results = pd.read_csv('OOD_Results - sentiment_analysis.csv')

# Keep first column and percentage columns and format for LaTeX
columns_to_keep = ['Unnamed: 0', 'BERT (%)', 'GPT2 (%)', 'T5 (%)', 'GPT4o (%)', 'CLAUDE (%)', 'LLAMA (%)']

# Function to format first column in bold and numbers with colors
def format_value(x, column):
    if column == 'Unnamed: 0':
        return f"\\textbf{{{x}}}"
    else:
        value = float(x)
        # Define color intensity based on value
        intensity = min(abs(value)/10, 1.0) # Scale intensity, max at 10% difference
        if value > 0:
            return f"\\cellcolor{{green!{int(intensity*50)}}}{{$+${abs(value):.2f}}}"
        else:
            return f"\\cellcolor{{red!{int(intensity*50)}}}{{$-${abs(value):.2f}}}"

# Add midrules between categories
prev_category = None
formatted_rows = []
for _, row in ood_results.iloc[:-2].iterrows():
    category = row['Unnamed: 0'].split(':')[0]
    if prev_category and category != prev_category:
        formatted_rows.append({col: '\\midrule' if col == 'Unnamed: 0' else '' for col in columns_to_keep})
    formatted_row = {col: format_value(row[col], col) for col in columns_to_keep}
    formatted_rows.append(formatted_row)
    prev_category = category

ood_results_with_midrules = pd.DataFrame(formatted_rows)

latex_table = ood_results_with_midrules.to_latex(
    index=False,
    escape=False,
    caption="Out-of-Distribution Results for SENT (% Difference)", 
    label="tab:ood_coref"
)

# Save LaTeX table
with open('ood_senti_table.tex', 'w') as f:
    f.write(latex_table)


In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv('./results/coref/claude-3-5-sonnet-0shot-discourse_100.csv')
df.head()
print(len(df))

In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
from scipy import stats

# Calculate accuracies
df['original_acc'] = df['original_pred'] == df['original_label']
df['modified_acc'] = df['modified_pred'] == df['modified_label']

# Convert boolean to int for t-test
original_acc = df['original_acc'].astype(int)
modified_acc = df['modified_acc'].astype(int)

# Perform paired t-test
t_stat, p_value = stats.ttest_rel(original_acc, modified_acc, alternative='less')

print("Paired t-test results:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Create contingency table for McNemar's test
n11 = sum((df['original_pred'] == df['original_label']) & (df['modified_pred'] == df['modified_label']))  # Both correct
n10 = sum((df['original_pred'] == df['original_label']) & (df['modified_pred'] != df['modified_label']))  # Original correct, modified wrong
n01 = sum((df['original_pred'] != df['original_label']) & (df['modified_pred'] == df['modified_label']))  # Original wrong, modified correct  
n00 = sum((df['original_pred'] != df['original_label']) & (df['modified_pred'] != df['modified_label']))  # Both wrong

# Create contingency table
contingency_table = [[n11, n10],
                    [n01, n00]]

# Perform McNemar's test
mcnemar_result = mcnemar(contingency_table, exact=True)

print("\nMcNemar's test results:")
print(f"Statistic: {mcnemar_result.statistic:.4f}")
print(f"P-value: {mcnemar_result.pvalue:.4f}")
print("\nContingency table:")
print(f"Both correct (n11): {n11}")
print(f"Original correct, modified wrong (n10): {n10}")
print(f"Original wrong, modified correct (n01): {n01}")
print(f"Both wrong (n00): {n00}")

# Calculate and print accuracies and percentage difference
print('\nAccuracies:')
print(f"Original accuracy: {original_acc.mean():.3f}")
print(f"Modified accuracy: {modified_acc.mean():.3f}")
print(f"Percentage difference: {((modified_acc.mean() - original_acc.mean()) / original_acc.mean()) * 100:.1f}%")

# Perform binomial test
n_trials = len(df)
n_original_correct = sum(original_acc)
n_modified_correct = sum(modified_acc)

# Binomial test for original accuracy
binom_original = stats.binomtest(n_original_correct, n_trials, p=original_acc.mean(), alternative='less')
print("\nBinomial test results:")
print(f"Original vs random chance:")
print(f"p-value: {binom_original.pvalue:.4f}")

# Binomial test for modified accuracy
binom_modified = stats.binomtest(n_modified_correct, n_trials, p=original_acc.mean(), alternative='less')
print(f"Modified vs random chance:")
print(f"p-value: {binom_modified.pvalue:.4f}")

In [ ]:
df = pd.read_csv('./results/coref/llama-0shot-active_to_passive_100.csv')
df.head()
print(len(df))


# Calculate accuracies
df['original_acc'] = df['original_pred'] == df['original_label']
df['modified_acc'] = df['modified_pred'] == df['modified_label']

# Convert boolean to int for t-test
original_acc = df['original_acc'].astype(int)
modified_acc = df['modified_acc'].astype(int)

# Perform paired t-test
t_stat, p_value = stats.ttest_rel(original_acc, modified_acc, alternative='greater')

print("Paired t-test results:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Create contingency table for McNemar's test
n11 = sum((df['original_pred'] == df['original_label']) & (df['modified_pred'] == df['modified_label']))  # Both correct
n10 = sum((df['original_pred'] == df['original_label']) & (df['modified_pred'] != df['modified_label']))  # Original correct, modified wrong
n01 = sum((df['original_pred'] != df['original_label']) & (df['modified_pred'] == df['modified_label']))  # Original wrong, modified correct  
n00 = sum((df['original_pred'] != df['original_label']) & (df['modified_pred'] != df['modified_label']))  # Both wrong

# Create contingency table
contingency_table = [[n11, n10],
                    [n01, n00]]

# Perform McNemar's test
mcnemar_result = mcnemar(contingency_table, exact=True)

print("\nMcNemar's test results:")
print(f"Statistic: {mcnemar_result.statistic:.4f}")
print(f"P-value: {mcnemar_result.pvalue:.4f}")
print("\nContingency table:")
print(f"Both correct (n11): {n11}")
print(f"Original correct, modified wrong (n10): {n10}")
print(f"Original wrong, modified correct (n01): {n01}")
print(f"Both wrong (n00): {n00}")

# Calculate and print accuracies and percentage difference
print('\nAccuracies:')
print(f"Original accuracy: {original_acc.mean():.3f}")
print(f"Modified accuracy: {modified_acc.mean():.3f}")
print(f"Percentage difference: {((modified_acc.mean() - original_acc.mean()) / original_acc.mean()) * 100:.1f}%")

# Perform binomial test
n_trials = len(df)
n_original_correct = sum(original_acc)
n_modified_correct = sum(modified_acc)

# Binomial test for original accuracy
binom_original = stats.binomtest(n_original_correct, n_trials, p=original_acc.mean())
print("\nBinomial test results:")
print(f"Original vs random chance:")
print(f"p-value: {binom_original.pvalue:.4f}")

# Binomial test for modified accuracy
binom_modified = stats.binomtest(n_modified_correct, n_trials, p=original_acc.mean())
print(f"Modified vs random chance:")
print(f"p-value: {binom_modified.pvalue:.4f}")

# Perform Wilcoxon signed-rank test
wilcoxon_result = stats.wilcoxon(original_acc, modified_acc, alternative='two-sided')

print("\nWilcoxon signed-rank test results:")
print(f"Statistic: {wilcoxon_result.statistic:.4f}")
print(f"P-value: {wilcoxon_result.pvalue:.4f}")


In [ ]:
df = pd.read_csv('./results/dialogue/claude-3-5-sonnet-0shot-geographical_bias_100.csv')
df.head()
print(len(df))


# Calculate accuracies
df['original_acc'] = df['original_pred'] == df['original_label']
df['modified_acc'] = df['modified_pred'] == df['modified_label']

# Convert boolean to int for t-test
original_acc = df['original_acc'].astype(int)
modified_acc = df['modified_acc'].astype(int)

# Perform paired t-test
t_stat, p_value = stats.ttest_rel(original_acc, modified_acc, alternative='two-sided')

print("Paired t-test results:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Create contingency table for McNemar's test
n11 = sum((df['original_pred'] == df['original_label']) & (df['modified_pred'] == df['modified_label']))  # Both correct
n10 = sum((df['original_pred'] == df['original_label']) & (df['modified_pred'] != df['modified_label']))  # Original correct, modified wrong
n01 = sum((df['original_pred'] != df['original_label']) & (df['modified_pred'] == df['modified_label']))  # Original wrong, modified correct  
n00 = sum((df['original_pred'] != df['original_label']) & (df['modified_pred'] != df['modified_label']))  # Both wrong

# Create contingency table
contingency_table = [[n11, n10],
                    [n01, n00]]

# Perform McNemar's test
mcnemar_result = mcnemar(contingency_table, exact=True)

print("\nMcNemar's test results:")
print(f"Statistic: {mcnemar_result.statistic:.4f}")
print(f"P-value: {mcnemar_result.pvalue:.4f}")
print("\nContingency table:")
print(f"Both correct (n11): {n11}")
print(f"Original correct, modified wrong (n10): {n10}")
print(f"Original wrong, modified correct (n01): {n01}")
print(f"Both wrong (n00): {n00}")

# Calculate and print accuracies and percentage difference
print('\nAccuracies:')
print(f"Original accuracy: {original_acc.mean():.3f}")
print(f"Modified accuracy: {modified_acc.mean():.3f}")
print(f"Percentage difference: {((modified_acc.mean() - original_acc.mean()) / original_acc.mean()) * 100:.1f}%")

# Perform binomial test
n_trials = len(df)
n_original_correct = sum(original_acc)
n_modified_correct = sum(modified_acc)

# Binomial test for original accuracy
binom_original = stats.binomtest(n_original_correct, n_trials, p=original_acc.mean())
print("\nBinomial test results:")
print(f"Original vs random chance:")
print(f"p-value: {binom_original.pvalue:.4f}")

# Binomial test for modified accuracy
binom_modified = stats.binomtest(n_modified_correct, n_trials, p=original_acc.mean(), alternative='two-sided')
print(f"Modified vs random chance:")
print(f"p-value: {binom_modified.pvalue:.4f}")

# Perform Wilcoxon signed-rank test
wilcoxon_result = stats.wilcoxon(original_acc, modified_acc, alternative='two-sided')

print("\nWilcoxon signed-rank test results:")
print(f"Statistic: {wilcoxon_result.statistic:.4f}")
print(f"P-value: {wilcoxon_result.pvalue:.4f}")


In [44]:
import pandas as pd
import json
import ast
# Read the CSV file
df = pd.read_csv('../data/modified_data/coref/active_to_passive_fix.csv')

# Read the JSON file
with open('../data/modified_data/coref/active_to_passive_100.json', 'r') as f:
    data = json.load(f)

# Create mapping of indices to data entries
# data_dict = {item['index']: item for item in data}

# Update data based on filtered rows
for idx, row in df.iterrows():
    # Update the entry with new values from CSV
    data[idx]['original_label'] = int(row['original_label'])
    data[idx]['modified_label'] = int(row['modified_label'])
    data[idx]['original_pronoun'] = row['original_pronoun']
    data[idx]['modified_pronoun'] = row['modified_pronoun']
    data[idx]['original_candidates'] = ast.literal_eval(row['original_candidates'])
    data[idx]['modified_candidates'] = ast.literal_eval(row['modified_candidates'])

# Convert back to list and write to file
updated_data = data
with open('../data/modified_data/coref/active_to_passive_100.json', 'w') as f:
    json.dump(updated_data, f, indent=2)


In [67]:
import pandas as pd
import json
import ast
# Read the CSV file
df = pd.read_csv('../data/modified_data/coref/length_bias_fix.csv')

# Read the JSON file
with open('../data/modified_data/coref/length_bias_100.json', 'r') as f:
    data = json.load(f)

# Create mapping of indices to data entries
# data_dict = {item['index']: item for item in data}

# Update data based on filtered rows
for idx, row in df.iterrows():
    # Update the entry with new values from CSV
    data[idx]['modified_label'] = int(row['modified_label'])
    data[idx]['modified_pronoun'] = row['modified_pronoun']
    data[idx]['modified_candidates'] = ast.literal_eval(row['modified_candidates'])
    data[idx]['modified_text'] = row['modified_text']
    

# Convert back to list and write to file
updated_data = data
with open('../data/modified_data/coref/length_bias_100.json', 'w') as f:
    json.dump(updated_data, f, indent=2)

In [51]:
import pandas as pd
import json
import ast
# Read the CSV file
df = pd.read_csv('../data/modified_data/dialogue/sentiment_100_fix.csv')

# Read the JSON file
with open('../data/modified_data/dialogue/sentiment_100.json', 'r') as f:
    data = json.load(f)
# Update data based on filtered rows
for idx, row in df.iterrows():
    # Update the entry with new values from CSV
    data[idx][1]['modified_label'] = int(row['modified_label'])

# Convert back to list and write to file
updated_data = data
with open('../data/modified_data/dialogue/sentiment_100.json', 'w') as f:
    json.dump(updated_data, f, indent=2)

In [ ]:
import pandas as pd
import json
import ast
# Read the CSV file
df = pd.read_csv('../data/modified_data/dialogue/grammatical_role_100_fix.csv')

# Read the JSON file
with open('../data/modified_data/dialogue/sentiment_100.json', 'r') as f:
    data = json.load(f)
# Update data based on filtered rows
for idx, row in df.iterrows():
    # Update the entry with new values from CSV
    data[idx][1]['modified_label'] = int(row['modified_label'])

# Convert back to list and write to file
updated_data = data
with open('../data/modified_data/dialogue/sentiment_100.json', 'w') as f:
    json.dump(updated_data, f, indent=2)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import json
import os
import glob

# Get all *_100.json files in coref's directory except excluded ones
excluded = []
json_files = []
for f in glob.glob('../data/modified_data/coref/*_100.json'):
    filename = os.path.basename(f).replace('_100.json','')
    if filename not in excluded:
        json_files.append(f)

for file_path in json_files:
    print(f"Processing {file_path}")
    
    # Read JSON file
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    modified = False
    # Track seen entries to remove duplicates
    seen_entries = set()
    filtered_data = []
    
    for entry in data:
        # Create tuple of key fields to check for duplicates
        entry_key = (entry['original_text'], entry['modified_text'], 
                    tuple(entry['original_candidates']), tuple(entry['modified_candidates']),
                    entry['original_pronoun'], entry['modified_pronoun'])
        
        if entry_key not in seen_entries:
            seen_entries.add(entry_key)
            
            # Check if modified pronoun exists in modified text
            # if entry['modified_pronoun'] not in entry['modified_text']:
            #     print(f"Missing pronoun {entry['modified_pronoun']} in modified text: {entry['modified_text']}")
            #     modified = True
                
            # # Check if modified candidates exist in modified text
            # for candidate in entry['modified_candidates']:
            #     if candidate not in entry['modified_text']:
            #         print(f"Missing candidate {candidate} in modified text: {entry['modified_text']}")
            #         modified = True
                    
            # Reset modified label if it differs from original
            if entry['modified_label'] != entry['original_label']:
                print(f"Label mismatch in: {entry['modified_text']}")
                # entry['modified_label'] = entry['original_label']
                modified = True
                
            filtered_data.append(entry)
        else:
            print(f"Found duplicate entry: {entry['modified_text']}")
            modified = True
    
    # Replace data with deduplicated version
    data[:] = filtered_data
    # Save changes if modifications were made
    if modified:
        print(f"Modified {file_path}")
        print('===================')
        # with open(file_path, 'w') as f:
        #     json.dump(data, f, indent=2)


In [ ]:
import pandas as pd
import glob
import os

# Get all CSV files matching the pattern
csv_files = glob.glob('results/ner/*-0shot-*_100_new.csv')

def parse_entities(entities):
    if isinstance(entities, str):
        entities = ast.literal_eval(entities)
    parsed_entities = []
    for item in entities:
    
        if 'text' in item:
            label = item['value'] if 'value' in item else item['label']
            parsed_entities.append({'text': item['text'].strip(), 'label': label.strip()})
        else:
            for key, value in item.items():
                parsed_entities.append({'text': key.strip(), 'label': value.strip()})
    parsed_entities = sorted(parsed_entities, key=lambda x: x['text'])
    return parsed_entities

for file_path in csv_files:
    print(f"Processing {file_path}")
    
    # Read CSV file
    df = pd.read_csv(file_path)

    df['modified_label'] = df['modified_label'].apply(parse_entities)
    df['original_label'] = df['original_label'].apply(parse_entities)
    df['modified_pred'] = df['modified_pred'].apply(parse_entities)
    df['original_pred'] = df['original_pred'].apply(parse_entities)

    # Save back to CSV
    df.to_csv(file_path.replace('_100_new.csv', '_100_compare.csv'), index=False)
    print(f"Saved formatted {file_path}")



In [ ]:
# Read all results files
files_and_tasks = {
    '../pretrained_language_models/ner/ner_modification_results_combined.csv': 'NER',
    '../pretrained_language_models/coreference_resolution/tmp/coreference_combined_results.csv': 'Coreference', 
    '../pretrained_language_models/dialogue_contradiction_detection/tmp/dialogue_combined_results.csv': 'Dialogue',
    '../pretrained_language_models/sentiment_analysis/sentiment_analysis_results.csv': 'Sentiment'
}

# Read and combine all dataframes
dfs = []
for file_path, task in files_and_tasks.items():
    df = pd.read_csv(file_path)
    if 'mean_f1_pct_change' in df.columns:
        df = df.rename(columns={'mean_f1_pct_change': 'pct_difference'})
    if 'percentage_diff' in df.columns:
        df = df.rename(columns={'percentage_diff': 'pct_difference'})
    if 'pct_diff' in df.columns:
        df = df.rename(columns={'pct_diff': 'pct_difference'})
    df['task'] = task
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
# Keep only the specified columns
combined_df = combined_df[['model', 'task', 'modification', 'pct_difference']]

# Normalize model names
def normalize_model_name(model):
    model = model.lower()
    if 'bert' in model:
        return 'bert'
    elif 'gpt-2' in model or 'gpt2' in model:
        return 'gpt2'
    elif 't5' in model:
        return 't5'
    elif 'gpt4' in model or 'gpt-4' in model:
        return 'gpt4o'
    elif 'claude' in model:
        return 'claude'
    elif 'llama' in model:
        return 'llama'
    return model
print(combined_df['model'].unique())
combined_df['model'] = combined_df['model'].apply(normalize_model_name)
print(combined_df['model'].unique())
# Take absolute value of pct_difference
combined_df['abs_pct_diff'] = combined_df['pct_difference'].abs()
combined_df.to_csv('combined_df.csv', index=False)
# Create pivot table with modifications as rows and models as columns
# Using mean since we want to see average impact across all tasks
final_df = pd.pivot_table(combined_df, 
                         values='abs_pct_diff',
                         index='modification',
                         columns='model',
                         aggfunc='mean').round(4)

# Add mean across models for each modification
final_df['average'] = final_df.mean(axis=1).round(4)

# Add mean across modifications for each model
model_avgs = final_df.mean().round(4)
final_df.loc['avg_across_mods'] = model_avgs

# Create LaTeX table
mod_mapping = {
    'temporal_bias': 'B: Tem',
    'geographical_bias': 'B: Geo', 
    'length_bias': 'B: Len',
    'typo_bias': 'O: Spell',
    'capitalization': 'O: Cap',
    'punctuation': 'O: Punc',
    'derivation': 'M: Deri',
    'compound_word': 'M: Com',
    'active_to_passive': 'Sx: Voice',
    'grammatical_role': 'Sx: Gra',
    'coordinating_conjunction': 'Sx: Conj',
    'concept_replacement': 'Sm: Con',
    'negation': 'P: Neg',
    'discourse': 'P: Disc',
    'sentiment': 'P: Senti',
    'casual': 'G: Cas',
    'dialectal': 'G: Dial'
}

model_order = ['bert', 'gpt2', 't5', 'gpt4o', 'claude', 'llama', 'average']

# Reorder and rename modifications
final_df = final_df[model_order]
final_df.index = [mod_mapping.get(idx, idx) for idx in final_df.index]

latex_table = "\\begin{table}[t]\n\\centering\n\\begin{tabular}{l|" + "c"*len(model_order) + "}\n"
latex_table += "\\hline\nModification & " + " & ".join(model_order) + " \\\\\n\\hline\n"

current_prefix = None

# Get max values for last column and row for scaling
avg_col_max = final_df['average'].max()
avg_row_max = final_df.loc['avg_across_mods'].max()

for idx in mod_mapping.values():
    if idx in final_df.index:
        # Add hline when prefix changes
        prefix = idx.split(':')[0]
        if current_prefix is not None and prefix != current_prefix:
            latex_table += "\\hline\n"
        current_prefix = prefix
        
        # Bold the modification name
        bold_idx = f"\\textbf{{{idx}}}"
        row = []
        
        for col in model_order:
            val = final_df.loc[idx, col]
            if col == 'average':  # For the last column
                intensity = int((val / avg_col_max) * 30)  # Scale based on average column max
                row.append(f"\\cellcolor{{brown!{intensity}}} {val:.2f}")
            else:
                row.append(f"{val:.2f}")
        latex_table += f"{bold_idx} & " + " & ".join(row) + " \\\\\n"

latex_table += "\\hline\n"
avg_row = []
for col in model_order:
    val = final_df.loc['avg_across_mods', col]
    if col == 'average':
        intensity = int((val / avg_row_max) * 30)  # Scale based on average row max
        avg_row.append(f"\\cellcolor{{yellow!{intensity}}} {val:.2f}")
    else:
        intensity = int((val / avg_row_max) * 30)  # Scale based on average row max
        avg_row.append(f"\\cellcolor{{purple!{intensity}}} {val:.2f}")
latex_table += "\\textbf{Average} & " + " & ".join(avg_row) + " \\\\\n"
latex_table += "\\hline\n\\end{tabular}\n"
latex_table += "\\caption{Average absolute percentage change in performance across different modifications and models}\n"
latex_table += "\\label{tab:modifications}\n\\end{table}"
print(latex_table)


In [ ]:
(4.7+3.6+2.4+5.6)/4